#### Idei:
* x Overfitting: schimbat dropout
* x Fixed Q target
* x Double DNN
* x O mica recompensa negativa pentru STILL_ALIVE, poate il va determina sa faca mai putini pasi random


* Normalizare date
* Adauga limita maxima de timp pentru online training. De exemplu, dupa 50s (fara schimbare de scor?), toate mutarile devin DOWN, ca sa iasa din tabla.
* Dueling DQN
* Gamma mai mare
* Testat diferite modele si imbunatatiri
* Considerat invatare online doar pe ultimele exemple (max memory = 32)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
import io
from PIL import Image, ImageOps
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob
from collections import deque

set_image_data_format('channels_first')

Using TensorFlow backend.


#### Parameters

In [32]:
STILL_ALIVE_REWARD = -0.01
STILL_ALIVE = -0.05
DEAD_REWARD = -1
SCORE_REWARD = 1
REWARD_POSITION = 3

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 100
RESIZE_HEIGHT = 71
READ_BATCH = 2

MOVES = 4
RIGHT = 0
DOWN = 1
LEFT = 2
UP = 3

SHOTS_FOLDER = 'data/shots/'
EXP_FOLDER = 'data/experience/'

SAMPLE_ALIVE = 1
SAMPLE_DEAD = 5
SAMPLE_REWARD = 20

MAX_MEMORY = 500
MAX_PLAY = 30

GAMMA = 0.9
LEARNING_RATE = 0.0001
EPS_INIT = 1.0
EPS_MIN = 0.05
EPS_DECAY = 0.998
CLONE_STEPS = 3000

#### Browser functions

In [3]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard(board):        
    try:
        board.click()
    except WebDriverException:
        print('Exception')
        return
    
    
def saveScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getScreen():
    ss = browser.get_screenshot_as_png()
    im = Image.open(io.BytesIO(ss))
    
    im = preprocImg(im)
    return np.asarray(im.convert("L"))
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def getBoard():
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)
    
    return board


def makeMove(board, m):    
    
    if m == RIGHT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == DOWN:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == LEFT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == UP:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

#### Preprocess functions

In [4]:

def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)


def saveImage(arr, file):
    '''Save an image array to a file'''
    img = Image.fromarray(arr.astype(np.uint8))
    img.save(file)
        

def preprocImg(im):
    '''Preprocess a screenshot.'''     
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = Image.open(fileName)   
            im = preprocImg(im)
            im.save(fileName)
            

### Q model

#### DNN action output

Functional API, citeste batch-uri de imagini, considera actiunea ca output. 

In [5]:

def nn_model_actionoutput():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
#     conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(32, (4, 4), activation='relu', name='conv3')(pool)
#     conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.0, name='dropout1')(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.0, name='dropout2')(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    optimizer = Adam(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


def nn_model_nopool():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.
    Does not use pooling layers, only strides.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), strides=(2, 2), activation='relu', name='conv1')(screen_input)    
    conv = Conv2D(32, (4, 4), strides=(2, 2), activation='relu', name='conv2')(conv)    
    conv = Conv2D(16, (4, 4), strides=(2, 2), activation='relu', name='conv3')(conv)
    
    flat = Flatten(name='flat')(conv)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.0, name='dropout1')(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.0, name='dropout2')(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    optimizer = Adam(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


def nn_model_deepmind():
    '''Full NN model, using the Keras Functional API.
    Previous action as input.
    Action as output, final layer has as many neurons as possible actions.
    The structure of the NN is based on the one used by DeepMind.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(32, (8, 8), strides=(4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(64, (4, 4), strides=(2, 2), activation='relu', name='conv2')(conv)
    conv = Conv2D(64, (3, 3), activation='relu', name='conv3')(conv)
    
    flat = Flatten(name='flat')(conv)
    dense = Dense(512, activation='relu', name='dense')(flat)   
    
    action_input = Input(shape=(1,), name='action_input')
    concat = keras.layers.concatenate([flat, action_input])
    
    output = Dense(MOVES, activation='linear', name='output')(concat)
    
    model = Model(inputs=[screen_input, action_input], outputs=output)
    
    optimizer = RMSprop(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


#### Experience class and functions

In [37]:

class Experience:
    '''Class for an experience.'''    
    
    def __init__(self, state=[], prevAction=0, action=0, label=0, nextState=[]):
        self.state = state
        self.prevAction = prevAction
        self.action = action
        self.label = label
        self.nextState = nextState
        
        
    def save(self, folder, index):
        '''Save experience to a file.'''

        file = folder + str(index).zfill(4) + '_' + str(self.prevAction) + '_' + str(self.action) + '_' + str(self.label) + '.xp'
        f = open(file, 'w')

        for s in self.state:
            list = s.flatten()
            for l in list:
                f.write(str(l) + ' ')
            f.write('\n')

        for s in self.nextState:
            list = s.flatten()
            for l in list:
                f.write(str(l) + ' ')
            f.write('\n')

        f.close()


    def readArray(self, line):
        '''Read an array representing an screenshot from a file.'''
        elems = line.split(' ')[:-1]

        a = []
        for e in elems:
            a.append(int(e))

        a = np.asarray(a)
        a = a.reshape(RESIZE_HEIGHT, RESIZE_WIDTH)

        return a

    
    def read(self, folder, file):
        '''Read an experience from a file.'''
        f = open(folder + file, 'r')

        elems = parseExpName(file)
        self.prevAction = int(elems[1])
        self.action = int(elems[2])
        self.label = float(elems[3])

        s1 = self.readArray(f.readline())
        s2 = self.readArray(f.readline())
        self.state = [s1, s2]

        ns1 = self.readArray(f.readline())
        ns2 = self.readArray(f.readline())
        self.nextState = [ns1, ns2]

        f.close()
                        
            
def rememberExp(memory, exps):
    '''Add new experiences to the memory.'''
    for e in exps:
        if len(memory) == MAX_MEMORY:
            memory.pop(0)
        memory.append(e)


def sampleMemory(memory, nSamples, propD, propR, propA):
    '''Sample experiences from the memroy according to some proportions.
    
    Parameters
    ----------
    memory: the remembered experiences
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    nSamples = min(len(memory), nSamples)
        
    perc = []
    nd, na, nr = (0, 0, 0)
    for exp in memory:
        r = float(parseExpName(f)[REWARD_POSITION])
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(memory, size=nSamples, replace=False, p=perc)
    return samples


def parseExpName(file):
    return file[:-3].split('_')


def sampleFiles(nSamples,
                folder=EXP_FOLDER,
                propD=SAMPLE_DEAD, 
                propR=SAMPLE_REWARD,
                propA=SAMPLE_ALIVE):
    '''Sample the experience files according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    folder: the experience folder
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(folder)
    nSamples = min(nSamples, len(files))
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = float(parseExpName(f)[REWARD_POSITION])
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
    print(perc)
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamples(samples):
    '''Read samples from experience files.'''
    states = []
    prevActions = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):    
        exp = Experience()
        exp.read(EXP_FOLDER, file)
            
        states.append(exp.state)
        prevActions.append(exp.prevAction)
        actions.append(exp.action)
        labels.append(exp.label)
        nextStates.append(exp.nextState)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    prevActions = np.asarray(prevActions)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
    
    return states, prevActions, actions, labels, nextStates


def getLastExpIndex(folder):
    '''Get the index of the last experience.'''
    files = os.listdir(folder)
    if len(files) == 0:
        return 0

    last = files[-1]
    index = parseExpName(last)[0]
    
    return int(index)


In [39]:
sampleFiles(20, propA=0, propD=1, propR=1)

[0.0023752969121140144, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0023752969121140144, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0023752969121140144, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0023752969121140144, 0.0, 0.0023752969121140144, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0023752969121140144, 0.0, 0

array(['1973_1_1_-1.xp', '1497_1_1_1.xp', '1635_2_1_1.xp',
       '3490_2_3_1.xp', '4031_1_3_1.xp', '0096_1_2_1.xp', '1720_1_3_1.xp',
       '4701_3_1_1.xp', '3593_0_3_-1.xp', '4663_1_1_-1.xp',
       '2335_3_1_1.xp', '5293_2_1_-1.xp', '5032_1_2_1.xp',
       '3131_1_1_-1.xp', '6266_2_0_-1.xp', '0412_0_0_1.xp',
       '4156_1_2_1.xp', '3453_3_1_-1.xp', '2512_0_0_-1.xp',
       '3805_2_0_1.xp'], dtype='<U17')

#### Train model function

In [7]:

def evaluateLabels(model, states, prevActions, actions, labels, sample=0):
    
    if sample > 0:
        sample = min(sample, len(states))
        indexes = random.sample(range(len(states)), sample)
        states = states[indexes]
        prevActions = prevActions[indexes]
        actions = actions[indexes]
        labels = labels[indexes]
        
    res = model.predict([states, prevActions])   
    
    sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
    meanSqErr = np.mean(sqErr)
    
    return meanSqErr

    
def moveString(m):
    if m == RIGHT:
        return 'right'
    elif m == DOWN:
        return 'down '
    elif m == LEFT:
        return 'left '
    elif m == UP:
        return 'up   '
    else:
        return 'none '
    
    
def getMoveOutputEps(model, screens, prevAction, eps):
    '''Get the move from the model which produces move scores as outputs, with an epsilon probability for exploration.'''
    
    # If there is a model
    if model != False:        
        p = random.random()
        
        # If exploitation was chosen
        if p > eps:
            res = model.predict([np.asarray([screens]), np.asarray([prevAction])])
            m = np.argmax(res)
            print('Model:', moveString(m), end='\r')

            return m
   
    # If there is no model or exploration was chosen
    m = int(MOVES * random.random())
    print('Rando:', moveString(m), end='\r')
    
    return m 


def copyModel(model, func):
    '''Copy a model and its weights.'''
    newModel = func()
    newModel.set_weights(model.get_weights())
    
    return newModel
    

def trainModelOnline(memory, 
                     model,
                     targetModel,
                     batchSize=32, 
                     batches=1,
                     file='data/models/model.h5',
                     status=True,
                     modelFunc=nn_model_actionoutput
                     ):
    '''Train a model online, from the experience memory. 
    Saves the model at the end.
    
    Parameters
    ----------
    memory: the experience memory
    model: the model to be trained
    targetModel: the fixed Q target model
    batchSize: the training batch size
    batches: number of batches to train on
    file: the model file, used for saving
    status: if true, print the status of the training
    modelFunc: the type of model to use, given by the model creation function
    '''
    
    global trainCount
        
    # Sample experiences from the memory
    samples = sampleMemory(memory, batches * batchSize, SAMPLE_DEAD, SAMPLE_REWARD, SAMPLE_ALIVE)
    batchSize = min(batchSize, len(samples))
    
    # For each batch
    bStart = 0
    
    while bStart < len(samples):
        
        bStop = min(bStart + batchSize, len(samples))
    
        states = []
        prevActions = []
        targets = []
        
        # For each experience
        while bStart < bStop: 
            e = samples[bStart]
            s = e.state
            p = e.prevAction
            a = e.action
            target = e.label
            n = np.asarray([e.nextState])
            
            if target != DEAD_REWARD:
                if target == STILL_ALIVE:
                    target = STILL_ALIVE_REWARD     
                    
                # Fixed Q target and Double DQN       
                bestAction = 0
                bestNextAction = - np.Infinity
                for m in range(MOVES):
                    action = np.argmax(model.predict([n, np.asarray([m])]))
                    if action > bestNextAction:
                        bestNextAction = action
                        bestAction = m
                
                target = target + GAMMA * targetModel.predict([n, np.asarray([bestAction])])[0][bestNextAction]

            fullTarget = model.predict([np.asarray([s]), np.asarray([p])])
            fullTarget[0][a] = target

            targets.append(fullTarget[0])
            states.append(s)
            prevActions.append(p)
                        
            trainCount += 1
            # Clone target model
            if trainCount % CLONE_STEPS == 0:
                targetModel = copyModel(model, modelFunc)            
            
            bStart += 1

        # Fit the model with the batch
        targets = np.asarray(targets)    
        model.train_on_batch([states, prevActions], targets)
        
        bStart = bStop
        
    model.save(file)
                
    return model


def trainModelOffline(states, prevActions, actions, labels, nextStates, 
                      epochs, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=False,
                      status=True,
                      modelFunc=nn_model_actionoutput
                     ):
    '''Train a model offline. 
    Can load or generate a new model. Saves the model after each epoch.
    Implements Fixed Q-target.
    
    Parameters
    ----------
    states: the list of batches of images representing the current states
    prevActions: the list of previous actions taken
    actions: the list of actions taken
    labels: the list of rewards obtained
    nextStates: the list of batches of images representing the next states
    epochs: the number of training epochs
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    modelFunc: the type of model to use, given by the model creation function
    '''
    
    if load:
        model = load_model(file)
        print('Model loaded.')
    else:
        model = modelFunc()
        
    # Initialize the target model
    targetModel = copyModel(model, modelFunc)

    dataSize = len(states)
    
    # For each epoch
    for i in range(epochs):
        if status:
            print('Epoch', i+1)
        
        time1 = time.time()
    
        batchStart = 0
        batchEnd = min(batchSize, dataSize)
    
        # For each batch
        while batchStart < batchEnd:

            targets = []
            
            b = batchStart
            # For each experience of the batch, recalculate the targets according to the Q algorithm
            while b < dataSize and b < batchEnd:
                # Get the features of the experience
                s = states[b:b+1]
                p = prevActions[b]
                a = actions[b]
                l = labels[b]
                n = nextStates[b:b+1]
                
                target = l
                if l != DEAD_REWARD:
                    if l == STILL_ALIVE:
                        l = STILL_ALIVE_REWARD                    
                    
                    # Fixed Q target and Double DQN
                    
                    bestAction = 0
                    bestNextAction = - np.Infinity
                    for m in range(MOVES):
                        action = np.argmax(model.predict([n, np.asarray([m])]))
                        if action > bestNextAction:
                            bestNextAction = action
                            bestAction = m
                    
                    target = l + GAMMA * targetModel.predict([n, np.asarray([bestAction])])[0][bestNextAction]
                
                fullTarget = model.predict([s, np.asarray([p])])
                fullTarget[0][a] = target
                
                targets.append(fullTarget[0])
                
                # Clone target model
                if (i * len(states) + b) % CLONE_STEPS == 0:
                    targetModel = copyModel(model, modelFunc)
                
                b += 1
            
            if status and batchStart % (batchSize * 2) == 0:
                print(batchStart, '/', dataSize, end='\r')
                
            # Fit the model with the batch
            targets = np.asarray(targets)
            model.train_on_batch([states[batchStart:batchEnd], prevActions[batchStart:batchEnd]], targets)

            batchStart = batchEnd
            batchEnd = min(batchEnd + batchSize, dataSize)
          
        if status:  
            print(dataSize, '/', dataSize)
            print("  MSE = %.2f" % evaluateLabels(model, states, prevActions, actions, labels, sample=50))
        
        time2 = time.time()
        
        if status:
            print("  Duration: %.2f s" % (time2- time1))
            
        # Save model every 10 epochs
        if i % 2 == 0:
            model.save(file) 
            
      
    model.save(file) 
        
    return model

### Play game

In [8]:

def play(games,
         train=False,
         useModel=False,
         saveExp=False,
         modelFile='data/models/model.h5',
         saveFolder=EXP_FOLDER,
         modelFunc=nn_model_actionoutput):
    
    '''Makes a move every second tick. The previous action does not influence the current action.'''
    
    global browser
    global trainCount
    
    e = EPS_INIT
    
    expIndex = getLastExpIndex(saveFolder) + 1
    
    url = 'https://playsnake.org/'
#     browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
    browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
    browser.get(url) 
    memory = []
    
    # Load model
    if useModel == True:
        curModel = load_model(modelFile)
        # Initialize the target model
        targetModel = copyModel(curModel, modelFunc)
    elif train == True:
        curModel = modelFunc()
        # Initialize the target model
        targetModel = copyModel(curModel, modelFunc)
    else:
        curModel = False        

    trainCount = 0
    for g in range(games):

        score = 0
        screenIndex = 0
        time1 = time.time()
    
        episode = []        
        finalScore = 0

        # Select the level
        selectLevel(1)        
        while getState() == 'countdown':
            pass  
        board = getBoard()      

        clickBoard(board)
        screens = deque([], 4)
        moves = deque([], 3)
        moves.append(DOWN)
        
        index = 0
        
        while getState() == 'playing' or getState() == 'paused':
                
            # Get current screenshot
            screens.append(getScreen())            

            # Calculate the reward for the previous move
            r = STILL_ALIVE
            if getScore() - score > 0:
                r = SCORE_REWARD 
                # Reset time between rewards
                time1 = time.time()
                

            score = getScore()
            finalScore = max(finalScore, score)  
            
            if index > 1:
                
                # Form the current state
                state = [screens[-2], screens[-1]]
                
                if index > 2:
                    # Save experience
                    prevState = [screens[0], screens[1]]
                    exp = Experience(prevState, moves[0], moves[1], r, state)
                    episode.append(exp)  
                
                # Check if max time between rewards has passed
                time2 = time.time()  
                if time2 - time1 > MAX_PLAY:
                    # Just repeat the last move until ending the game
                    print('Death:', moveString(m), end='\r')
                
                else:                
                    # Get the current move
                    m = getMoveOutputEps(curModel, state, moves[-1], e)
                    moves.append(m)
                      
            
                # Send the move to the board
                clickBoard(board)
                makeMove(board, m)
                clickBoard(board) 
                
            else:
                clickBoard(board)
                clickBoard(board)                 

            index += 1
            
        # Add last experience
        if len(episode) > 1:
            state = [screens[-1], screens[-1]]
            prevState = [screens[1], screens[2]]
            exp = Experience(prevState, moves[0], moves[1], DEAD_REWARD, state)
            episode.append(exp)   
            
        print("%d: %d points " % (g, finalScore))        
        
        # Save experiences
        if saveExp:
            # Only save experiences with positive rewards
            hasReward = 0
            for exp in episode:
                if exp.label > 0:
                    hasReward = 1
                    break
            if hasReward:
                for exp in episode:
                    exp.save(saveFolder, expIndex)
                    expIndex += 1  
            
        if train:
            # Add the episode to the memory
            rememberExp(memory, episode)

            # Retrain the model            
            curModel = trainModelOnline(memory, model=curModel, targetModel=targetModel, 
                                        file=modelFile, batches=5, modelFunc=modelFunc)
        else:
            time.sleep(1)            
            
        if e > EPS_MIN:
            e = e * EPS_DECAY

    browser.quit()
    
    
    ### DEBUGGING
    return episode



In [18]:
# Train online
EPS_INIT = 1.0
EPS_MIN = 0.1
EPS_DECAY = 0.997
MAX_MEMORY = 5000
episode = play(1000,
               train=True,
               useModel=True,
               modelFunc=nn_model_deepmind,
               modelFile='data/models/model2_deepmind_negalive_lr0001_g9_o1280.h5',
               saveExp=True
              )

In [10]:
# Save images
episode = play(200,
               saveExp=True   
              )

0: 0 points 
1: 0 points 
2: 0 points 
3: 0 points 
4: 0 points 
5: 0 points 
6: 5 points 
7: 0 points 
8: 0 points 
9: 0 points 
10: 0 points 
11: 0 points 
12: 0 points 
13: 0 points 
14: 0 points 
15: 0 points 
16: 0 points 
17: 0 points 
18: 55 points 
19: 0 points 
20: 0 points 
21: 0 points 
22: 0 points 
23: 0 points 
24: 0 points 
25: 0 points 
26: 0 points 
27: 0 points 
28: 0 points 
29: 0 points 
30: 0 points 
31: 0 points 
32: 0 points 
33: 0 points 
34: 0 points 
35: 0 points 
36: 0 points 
37: 0 points 
38: 0 points 
39: 5 points 
40: 5 points 
41: 5 points 
42: 0 points 
43: 0 points 
44: 0 points 
45: 15 points 
46: 0 points 
47: 0 points 
48: 0 points 
49: 0 points 
50: 0 points 
51: 0 points 
52: 0 points 
53: 5 points 
54: 0 points 
55: 0 points 
56: 5 points 
57: 0 points 
58: 0 points 
59: 0 points 
60: 0 points 
61: 0 points 
62: 95 points 
63: 0 points 
64: 0 points 
65: 0 points 
66: 0 points 
67: 0 points 
68: 0 points 
69: 65 points 
70: 0 points 
71: 0 points

KeyboardInterrupt: 

In [20]:
# Play with model
EPS_INIT = 0.0
episode = play(1,
               useModel=True, 
               modelFile='data/models/test.h5',
               modelFunc=nn_model_deepmind,
#                saveExp=True
#                train=True
              )

In [10]:
# Train offline
for i in range(1):
    states, prevActions, actions, labels, nextStates = readSamples(sampleFiles(3000))
    # s2, a2, l2, n2 = readSamples(sampleFiles(50))
    model = trainModelOffline(states, prevActions, actions, labels, nextStates,
                              modelFunc=nn_model_deepmind,
                              epochs=10, 
                              load=False, 
                              file='data/models/model2_deepmind_-01alive_lr0001_g9_e150_s3000.h5')

Sampled data.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1
Instructions for updating:
Use tf.cast instead.
3000 / 3000
  MSE = 35.61
  Duration: 36.02 s
Epoch 2
3000 / 3000
  MSE = 27.64
  Duration: 37.46 s
Epoch 3
3000 / 3000
  MSE = 30.30
  Duration: 39.13 s
Epoch 4
3000 / 3000
  MSE = 30.32
  Duration: 40.74 s
Epoch 5
3000 / 3000
  MSE = 32.29
  Duration: 43.06 s
Epoch 6
3000 / 3000
  MSE = 30.20
  Duration: 39.82 s
Epoch 7


KeyboardInterrupt: 

In [15]:
# del states, prevActions, actions, labels, nextStates
del model

NameError: name 'model' is not defined

In [16]:
import sys

# del states, prevActions, actions, labels, nextStates

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
s = sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
print(s)

NameError: name 'sys' is not defined

#### Evaluate

In [65]:
# Print model structure
from keras.utils.vis_utils import plot_model

model = load_model('data/models/model.h5')
plot_model(model, show_shapes=True, to_file='data/models/model.png')

In [12]:
for i, e in enumerate(episode):
    for j, s in enumerate(e.state):
        saveImage(s, 'data/shots2/e' + str(i) + '_c' + str(j) + '_' + moveString(e.prevAction) + '_' + moveString(e.action) + '_' + str(e.label) + '.png')
    for j, s in enumerate(e.nextState):
        saveImage(s, 'data/shots2/e' + str(i) + '_n' + str(j) + '.png')

In [53]:
# s2, a2, l2, n2 = readSamples(sampleFiles(50))

model = load_model('data/models/model_cnv3_lin_drp00_lr0001_e150_s3000_o2700.h5')

for i, state in enumerate(s2):
    saveImage(state[1], 'data/shots2/s' + str(i) + '.png')
#     print(model.predict(np.asarray([state])))